In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from datetime import datetime, timedelta

import warnings
warnings.filterwarnings('ignore', category=UserWarning, message='Glyph.*missing from font')

In [2]:
routes_df_source = pd.read_csv("csv-routes.csv")
routes_df = pd.DataFrame({
    "route_code": [
#        "2HM2+P8|XJV5+RG",     # Jaya Prakash Narayana Park → Coles Park, Fraser Town
        "VJRQ+2M|RMJJ+F4",     # Kudlu Gate Metro Station → Biocon Campus
        "WH5F+26|WJ8X+F5W",    # Jaya Prakash Nagar Metro Station → Hemavathi Park, HSR Layout
        "XJPW+92|WJP4+FF",     # Swami Vivekananda Road Metro Station → Christ University, Hosur Main Road
        "2HVW+G8|XJXR+WG",     # Bethel AG Church, Hebbal → SMVT Railway Station
        "WGG8+G5|XH7P+G6",     # The Watering Hole, Rajarajeshwari Nagar → Sir Puttanna Chetty Town Hall, Bangalore
        "XPC7+72|XM33+J3",     # The Rameshwaram Cafe @ Brookfield → Gawky Goose, Wind Tunnel Rd
        "WHCJ+26|XGCP+FV",     # RV Road Metro Station, Jayanagar 5th Block → Vijayanagar Metro Station, Chord Road
        "XMW9+G8|WMJR+V4",     # Benniganahalli Metro Station → Embassy TechVillage, Devarabisanahalli
#        "XHJ7+MG|WJM6+VC",     # Lulu Mall Bengaluru → Nexus Mall Koramangala
#        "WHR9+R6|XJGF+6J",     # Big Bull Temple, Basavanagudi → Shri Someshwara Swamy Temple, Halasuru
#        "WP44+W8|WJFH+XQ",     # Karmelaram Railway Station, Chikkabellandur → St. Francis College, Koramangala
#        "XJG4+7J|5PX4+HQ",     # MG Road Metro Station → Kempegowda International Airport, Bengaluru
]})

routes_df = routes_df.merge(routes_df_source, on=['route_code'], how='left')
display(routes_df)

,route_code,label_full,label_short
0,VJRQ+2M|RMJJ+F4,Kudlu Gate Metro Station → Biocon Campus,Hosur Road
1,WH5F+26|WJ8X+F5W,Jaya Prakash Nagar Metro Station → Hemavathi P...,Double Decker Flyover
2,XJPW+92|WJP4+FF,Swami Vivekananda Road Metro Station → Christ ...,East Inner Ring
3,2HVW+G8|XJXR+WG,"Bethel AG Church, Hebbal → SMVT Railway Station",North Outer Ring
4,WGG8+G5|XH7P+G6,"The Watering Hole, Rajarajeshwari Nagar → Sir ...",Mysore Road
5,XPC7+72|XM33+J3,The Rameshwaram Cafe @ Brookfield → Gawky Goos...,Old Airport Road
6,WHCJ+26|XGCP+FV,"RV Road Metro Station, Jayanagar 5th Block → V...",South Outer Ring
7,XMW9+G8|WMJR+V4,Benniganahalli Metro Station → Embassy TechVil...,East Outer Ring


![routes.png](routes.png)

In [3]:
locations = pd.read_csv("csv-locations_12.9514242_77.6590212.csv").to_dict(orient='list')
locate = lambda plus_code: locations['location'][locations['plus_code'].index(plus_code)]

source_df = pd.read_csv("csv-bangalore_traffic.csv")
# Keep only those rows of df that correspond to routes in routes_df
source_df = source_df[source_df['route_code'].isin(routes_df['route_code'])]
display(source_df)

,date,time,route_code,duration,distance
2,2025-09-25,14:25,VJRQ+2M|RMJJ+F4,23,10.3
3,2025-09-25,14:25,WH5F+26|WJ8X+F5W,25,10.2
4,2025-09-25,14:25,XJPW+92|WJP4+FF,38,10.4
5,2025-09-25,14:25,2HVW+G8|XJXR+WG,31,9.9
7,2025-09-25,14:25,XPC7+72|XM33+J3,27,9.4
...,...,...,...,...,...
3978,2025-10-09,18:38,XPC7+72|XM33+J3,43,10.6
3979,2025-10-09,18:38,2HVW+G8|XJXR+WG,41,9.9
3980,2025-10-09,18:38,WHCJ+26|XGCP+FV,46,10.2
3981,2025-10-09,18:38,XMW9+G8|WMJR+V4,48,10.1


In [4]:
def transformed_data(df_in):
    df_traffic = df_in.copy()
    df_traffic['year'] = pd.to_datetime(df_traffic['date']).dt.year
    df_traffic['month'] = pd.to_datetime(df_traffic['date']).dt.month
    df_traffic['day'] = pd.to_datetime(df_traffic['date']).dt.day
    df_traffic['hour'] = pd.to_datetime(df_traffic['time'], format='%H:%M', errors='coerce').dt.hour
    df_traffic['day_of_week'] = pd.to_datetime(df_traffic['date']).dt.day_name()
    df_traffic['avg_speed'] = round(df_traffic['distance'] / (df_traffic['duration'] / 60), 2)
#    df_traffic['origin'] = df_traffic['route_code'].str.split('|').str[0]
#    df_traffic['destination'] = df_traffic['route_code'].str.split('|').str[1]
#    df_traffic['origin'] = df_traffic['route_code'].str.split('|').str[0].apply(locate)
#    df_traffic['destination'] = df_traffic['route_code'].str.split('|').str[1].apply(locate)
    df_traffic = df_traffic[['year', 'month', 'day', 'hour', 'route_code', 'label_full', 'label_short', 'duration', 'distance', 'avg_speed']]
    df_traffic = df_traffic.sort_values(['year', 'month', 'day', 'hour', 'avg_speed'],\
                                ascending=[True, True, True, True, False]).reset_index(drop=True)
    return df_traffic

df = source_df.merge(routes_df, on='route_code')
df = transformed_data(df)
display(df)

,year,month,day,hour,route_code,label_full,label_short,duration,distance,avg_speed
0,2025,9,25,14,VJRQ+2M|RMJJ+F4,Kudlu Gate Metro Station → Biocon Campus,Hosur Road,23,10.3,26.87
1,2025,9,25,14,WH5F+26|WJ8X+F5W,Jaya Prakash Nagar Metro Station → Hemavathi P...,Double Decker Flyover,25,10.2,24.48
2,2025,9,25,14,XMW9+G8|WMJR+V4,Benniganahalli Metro Station → Embassy TechVil...,East Outer Ring,26,10.1,23.31
3,2025,9,25,14,XPC7+72|XM33+J3,The Rameshwaram Cafe @ Brookfield → Gawky Goos...,Old Airport Road,27,9.4,20.89
4,2025,9,25,14,2HVW+G8|XJXR+WG,"Bethel AG Church, Hebbal → SMVT Railway Station",North Outer Ring,31,9.9,19.16
...,...,...,...,...,...,...,...,...,...,...
2485,2025,10,9,18,WH5F+26|WJ8X+F5W,Jaya Prakash Nagar Metro Station → Hemavathi P...,Double Decker Flyover,42,10.2,14.57
2486,2025,10,9,18,2HVW+G8|XJXR+WG,"Bethel AG Church, Hebbal → SMVT Railway Station",North Outer Ring,41,9.9,14.49
2487,2025,10,9,18,WHCJ+26|XGCP+FV,"RV Road Metro Station, Jayanagar 5th Block → V...",South Outer Ring,46,10.2,13.30
2488,2025,10,9,18,XJPW+92|WJP4+FF,Swami Vivekananda Road Metro Station → Christ ...,East Inner Ring,53,11.3,12.79


In [5]:
display(df.describe().round(2))
display(df.describe(exclude='number'))
display(df['route_code'].value_counts())

,year,month,day,hour,duration,distance,avg_speed
count,2490.0,2490.00,2490.00,2490.00,2490.00,2490.00,2490.00
mean,2025.0,9.65,12.94,11.79,25.62,10.19,26.40
std,0.0,0.48,11.14,6.94,8.45,0.46,8.17
min,2025.0,9.00,1.00,0.00,13.00,9.20,11.02
25%,2025.0,9.00,4.00,5.00,19.00,9.90,19.36
50%,2025.0,10.00,7.00,12.00,24.00,10.20,25.64
75%,2025.0,10.00,27.00,18.00,32.00,10.40,32.53
max,2025.0,10.00,30.00,23.00,55.00,13.60,47.54


,route_code,label_full,label_short
count,2490,2490,2490
unique,8,8,8
top,VJRQ+2M|RMJJ+F4,Kudlu Gate Metro Station → Biocon Campus,Hosur Road
freq,327,327,327


route_code
VJRQ+2M|RMJJ+F4     327
WH5F+26|WJ8X+F5W    327
XMW9+G8|WMJR+V4     327
XPC7+72|XM33+J3     327
2HVW+G8|XJXR+WG     327
WHCJ+26|XGCP+FV     327
XJPW+92|WJP4+FF     327
WGG8+G5|XH7P+G6     201
Name: count, dtype: int64

In [6]:
df_minmax = df[(df['hour'] >= 0) & (df['hour'] <= 23)].copy()

df_minmax = df_minmax.groupby(['label_short', 'hour'])['avg_speed'].mean().unstack().reset_index()
df_minmax['variance'] = df_minmax.iloc[:, 1:].apply(lambda row: row.max() - row.min(), axis=1)
df_minmax['max_hour'] = df_minmax.iloc[:, 1:-1].apply(lambda row: row.idxmax(), axis=1)
df_minmax['min_hour'] = df_minmax.iloc[:, 1:-2].apply(lambda row: row.idxmin(), axis=1)

df_minmax[['label_short', 'max_hour', 'min_hour', 'variance']].\
    sort_values(by='variance', ascending=False).reset_index(drop=True)

hour,label_short,max_hour,min_hour,variance
0,East Inner Ring,3,18,22.155714
1,Old Airport Road,4,18,21.582238
2,Mysore Road,3,19,20.762222
3,Hosur Road,3,18,20.536286
4,South Outer Ring,3,18,19.563286
5,Double Decker Flyover,2,18,18.169429
6,North Outer Ring,4,18,15.321714
7,East Outer Ring,3,17,13.751048


In [ ]:
def plot_traffic_square(df_incoming, days_offset=1, height='square', dpi=300, label='short'):
    """
    Plot average speed over time for all routes in the raw DataFrame.
    
    Parameters
    ----------
    df_incoming : pd.DataFrame
        Must contain columns: 
        - 'date' (str, YYYY-MM-DD)
        - 'time' (str, HH:MM)
        - 'route' (str, human-readable route name)
        - 'duration' (numeric, minutes)
        - 'distance' (numeric, km)
    height : str, default='square'
        Figure size: 'square' (16, 16), 'wide' (22, 16), or 'extrawide' (28, 16)
        Hour interval: 'square' (1), 'wide' (3), or 'extrawide' (6)
    """
    if df_incoming.empty:
        print("No data available in the dataset.")
        return

    df_plot = df_incoming.copy()
    HOURS_OFFSET = (datetime.now() - timedelta(hours=int(days_offset * 24)))
    df_plot['ts'] = pd.to_datetime(df_plot[['year', 'month', 'day', 'hour']])
    df_plot = df_plot[pd.to_datetime(df_plot['ts']) >= HOURS_OFFSET]

    if label in ['short', 'full']:
        df_plot['route_label'] = df_plot['label_' + label]
    else:
        print("label must be 'short' or 'full', got: " + label)
        return

    display(df_plot)

    # Average speed (km/h) = 60 * distance / duration
#    df_plot['avg_speed'] = 60.0 * df_plot['distance'] / df_plot['duration'].replace(0, pd.NA)
        
    # 2) Build a common timeline (all observed timestamps)
    timeline = pd.Index(sorted(df_plot['ts'].unique()))
    
    # 3) Fill missing data and smooth per route
    def fill_and_smooth_route(df):
        """Reindex to timeline, fill missing speeds, and smooth."""
        g = df.set_index('ts').reindex(timeline)
        
        # Fill average speed with neighbor mean
        speeds = pd.to_numeric(g['avg_speed'], errors='coerce')
        prev_vals = speeds.ffill()
        next_vals = speeds.bfill()
        filled = speeds.copy()
        
        mask_missing = speeds.isna()
        mask_both = mask_missing & prev_vals.notna() & next_vals.notna()
        filled.loc[mask_both] = (prev_vals.loc[mask_both] + next_vals.loc[mask_both]) / 2.0
        
        mask_prev_only = mask_missing & prev_vals.notna() & next_vals.isna()
        mask_next_only = mask_missing & next_vals.notna() & prev_vals.isna()
        filled.loc[mask_prev_only] = prev_vals.loc[mask_prev_only]
        filled.loc[mask_next_only] = next_vals.loc[mask_next_only]
        
        g['speed_filled'] = filled
        
        # Smooth the filled speeds (ts is already in the index before reset)
        ts_series = g.index.to_series()
        speed_series = g['speed_filled']
        
        mask_valid = speed_series.notna()
        ts_valid = ts_series[mask_valid]
        speed_valid = speed_series[mask_valid]
        
        if len(speed_valid) < 3:
            g['speed_smooth'] = speed_series
        else:
            try:
                from scipy.interpolate import PchipInterpolator
                x = ts_valid.map(pd.Timestamp.toordinal).to_numpy(dtype=float)
                y = speed_valid.to_numpy(dtype=float)
                x_all = ts_series.map(pd.Timestamp.toordinal).to_numpy(dtype=float)
                interp = PchipInterpolator(x, y)
                g['speed_smooth'] = pd.Series(interp(x_all), index=g.index)
            except Exception:
                # Fallback: centered rolling mean
                win = 3 if len(speed_valid) < 10 else 5
                g['speed_smooth'] = speed_series.rolling(window=win, center=True, min_periods=1).mean()
        
        return g.reset_index(names='ts')

    # Process each route
    frames = []
    for route in sorted(df['route'].unique()):
        route_data = df.loc[df['route'] == route, ['ts', 'route', 'avg_speed']]
        frames.append(fill_and_smooth_route(route_data))
    
    df_filled = pd.concat(frames, ignore_index=True).sort_values(['ts', 'route'])

    # 4) Determine figure size based on height parameter
    figsize = ()
    if height == 'square':
        figsize = (14, 14)
        hour_interval = 1
        legend_fontsize = 12
    elif height == 'wide':
        figsize = (20, 14)
        hour_interval = 3
        legend_fontsize = 14
    elif height == 'extrawide':
        figsize = (26, 14)
        hour_interval = 6
        legend_fontsize = 16
    elif height == 'extrawide2':
        figsize = (30, 12)
        hour_interval = 6
        legend_fontsize = 18
    else:
        raise ValueError(f"height must be 'square', 'wide', or 'extrawide', got: {height}")
    
    # 5) Plot: average speed, legend outside, HH-only x-axis
    hue_order = sorted(df_filled['route'].dropna().unique())
    palette = sns.color_palette("tab20", n_colors=len(hue_order))
    
    plt.figure(figsize=figsize, dpi=300)
    ax = sns.lineplot(
        data=df_filled,
        x='ts', y='speed_smooth',
        hue='route',
        hue_order=hue_order,
        palette=palette,
        linewidth=8, 
        alpha=0.4)
    ax.grid(True, which='both', linestyle='--', linewidth=0.3)
    ax.margins(x=0)
    ax.set_xlim(df_filled['ts'].min(), df_filled['ts'].max())
    ax.tick_params(axis='x', labelsize=legend_fontsize)
    ax.tick_params(axis='y', labelsize=legend_fontsize)
    
    # X-axis as hours only (HH) with dynamic interval
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H'))
    ax.xaxis.set_major_locator(mdates.HourLocator(interval=hour_interval))
    ax.set_xlabel('Time (Hour)'.upper(), fontsize=legend_fontsize+2)
    ax.set_ylabel('Average Speed (km/h)'.upper(), fontsize=legend_fontsize+2)
    
    # Dynamic title based on time range
    end   = df_filled['ts'].max()
    start = df_filled['ts'].min()
    time_range_hours = (end - start).total_seconds() / 3600
    ax.set_title(f'Average Speeds Over Last {time_range_hours:.0f} Hours', fontsize=legend_fontsize+2)
    
    # Mark midnight with dark vertical lines and add day-of-week labels
    ylim = ax.get_ylim()
    
    # Find all midnight timestamps in the data range
    midnight_times = pd.date_range(
        start=start.normalize() + pd.Timedelta(days=1),
        end=end,
        freq='D')
    
    for midnight in midnight_times:
        # Draw dark vertical line at midnight
        ax.axvline(x=midnight, color='black', linewidth=1, linestyle='-', zorder=1)
        
        # Add day-of-week label at the top, just to the right of the midnight line
        day_name = midnight.strftime('%A')
        ax.text(
            midnight, ylim[1] * 0.98,  # just below top frame
            day_name,
            ha='left',
            va='top',
            fontsize=legend_fontsize,
            fontweight='bold',
            color='black')
    
    # Legend outside on the bottom:
    handles, labels = ax.get_legend_handles_labels()
    leg = ax.legend(
        handles, labels,
        loc='upper center',
        bbox_to_anchor=(0.5, -0.08),
        ncol=2,
        frameon=True,
        borderaxespad=0.0,
        fontsize=legend_fontsize,
        borderpad=1.0,
        labelspacing=0.8
    )
    leg.get_frame().set_linewidth(0.8)
    leg.get_frame().set_edgecolor('#000000')
    plt.subplots_adjust(bottom=0.24)
    plt.tight_layout()
    plt.show()
    
    return df

In [ ]:
def plot_route_boxplots(df_incoming, avg_speed=True, duration=True, legend=True):
    """
    Generate boxplots for route metrics.
    
    Parameters
    ----------
    df_incoming : pd.DataFrame
        Must contain columns: ts, route, duration, distance, avg_speed
    plot_type : str, default='both'
        Type of plot to generate:
        - 'speed': Average speed boxplot only
        - 'duration': Duration boxplot only
        - 'both': Both plots side-by-side (default)
    legend : bool, default=True
        If True, show route labels vertically below the x-axis.
        If False, hide x-axis labels.
    """
    df_box = df_incoming.copy()
    df_box['label'] = df_box['route']
    df_box.drop(['route'], axis=1, inplace=True)
    
    # Select metrics based on plot_type
    metrics = {}
    if avg_speed:
        metrics['avg_speed'] = 'Average speed (km/h)'
    if duration:
        metrics['duration'] = 'Duration (minutes)'
    if  metrics == {}:
        raise ValueError("At least one of avg_speed or duration must be True")

    # Sort routes by median avg_speed (descending) - use this order for all plots
    route_order = (
        df_box.groupby('label')['avg_speed']
              .median()
              .sort_values(ascending=False)
              .index.tolist()
    )

    # Create color map
    hue_order = sorted(df_box['label'].unique())
    palette = sns.color_palette("tab20", n_colors=len(hue_order))
    color_map = dict(zip(hue_order, palette))
    
    # Adjust figure size based on number of plots
    figsize = (12, 8) if len(metrics) == 1 else (16, 8)
    if legend:
        figsize = (figsize[0], figsize[1] + 4)
    
    fig, axes = plt.subplots(1, len(metrics), figsize=figsize, sharex=False, dpi=300)
    
    # Ensure axes is always iterable
    if len(metrics) == 1:
        axes = [axes]

    for ax, metric in zip(axes, metrics.keys()):
        sns.boxplot(
            data=df_box,
            x='label',
            y=df_box[metric],
            hue='label',
            order=route_order,
            hue_order=route_order,
            palette=color_map,
            dodge=False, 
            legend=False,
            ax=ax
        )
        
        # Make box faces semi-transparent
        for patch in ax.artists:
            fc = patch.get_facecolor()
            patch.set_facecolor((*fc[:3], 0.4))
        
        for line in ax.lines:
            line.set_alpha(0.4)
        
        ax.set_xlabel('')
        ax.set_ylabel(metrics[metric])
        ax.set_title(f'{metrics[metric]} by Route')
        
        # Control x-axis label visibility based on legend parameter
        if legend:
            ax.tick_params(axis='x', rotation=90, labelbottom=True, bottom=True)
        else:
            ax.tick_params(axis='x', labelbottom=False, bottom=False)
    
    plt.subplots_adjust(bottom=0.35)
    plt.tight_layout()
    plt.show()

In [ ]:
NUMBER_OF_DAYS = 1
NUMBER_OF_DAYS = (datetime.now() - timedelta(days=NUMBER_OF_DAYS)).strftime('%Y-%m-%d')
df_plot = df[df['date'] >= NUMBER_OF_DAYS].copy()

df_plot['route_code'] = df_plot['route_code'].\
    apply(lambda r: f"{locate(r.split('|')[0])} \u2192 {locate(r.split('|')[1])}")
df_plot = df_plot[['date', 'time', 'route_code', 'duration', 'distance']].\
    reset_index(drop=True)

display(df_plot.tail(12))

In [ ]:
plot_route_boxplots(
    plot_traffic_square(df_plot, height='square'), 
            avg_speed=True, duration=True, legend=False)

_____
## Rolling Relative Route Scoring System (The R³S² Score)

In [ ]:
def get_variances(df):
    df_variance = df.copy()
    df_variance['avg_speed'] = df_variance['distance'] / df_variance['duration']
    df_variance = df_variance\
           .groupby(['route_code'])[['duration', 'distance', 'avg_speed']]\
           .agg(['min', 'mean', 'max'])\
           .reset_index()
    df_variance = df_variance.sort_values(by=[('avg_speed', 'mean')], ascending=[False])\
           .reset_index(drop=True)
    df_variance[[('duration', 'mean'), ('distance', 'mean'), ('avg_speed', 'mean')]]\
           = df_variance[[('duration', 'mean'), ('distance', 'mean'), ('avg_speed', 'mean')]].round(2)
    df_variance['points'] = np.linspace(len(df_variance)/2, -len(df_variance)/2, len(df_variance))
    return df_variance[['route_code', 'points']]


def calculate_rrs(df, ref_date=None, DAYS_ROLLING=10):
    if ref_date is None:
        ref_date = datetime.now()
    else:
        ref_date = datetime.strptime(ref_date, '%Y-%m-%d')
    df_rrs = pd.DataFrame({'route_code': df['route_code'].unique().tolist(), 
                           'points': [0]*len(df['route_code'].unique().tolist())})
    df_rrs = df_rrs.sort_values(by=['route_code']).reset_index(drop=True)
    for d in range(DAYS_ROLLING, 0, -1):
        filter = (df['date'] == (ref_date - timedelta(days=d)).strftime('%Y-%m-%d'))
        df_points = get_variances(df[filter])
        df_points = df_points.sort_values(by=['route_code']).reset_index(drop=True)
        df_rrs['points'] += df_points['points']
    df_rrs = df_rrs.sort_values(by=['points'], ascending=False).round(1).reset_index(drop=True)
    return df_rrs

df_rrs = calculate_rrs(df, DAYS_ROLLING=10).dropna()
df_rrs['route'] = df_rrs['route_code'].map(lambda x: locate(x.split('|')[0]) + '\u2192' + locate(x.split('|')[1]))
df_rrs = df_rrs[['route', 'points']]

display(df_rrs)

In [ ]:
# Create a color gradient - green (best) to red (worst)
colors = plt.cm.RdYlGn_r(np.linspace(0.2, 0.8, len(df_rrs)))

# Create figure with dark grid for better readability
plt.figure(figsize=(14, 10), dpi=300)
ax = plt.gca()

# Create horizontal bar chart
bars = ax.barh(range(len(df_rrs)), df_rrs['points'], color=colors, edgecolor='black', linewidth=0.5)

# Add value labels on the bars
for i, (idx, row) in enumerate(df_rrs.iterrows()):
    value = row['points']
    # Position label inside or outside bar depending on value
    x_pos = value - (abs(value) * 0.05) if value > 0 else value + (abs(value) * 0.05)
    ax.text(x_pos, i, f'{value}', ha='right' if value > 0 else 'left', 
            va='center', fontweight='normal', fontsize=12, color='#333333')

# Set y-axis labels (inverted so rank 1 is on top)
ax.set_yticks(range(len(df_rrs)))
ax.set_yticklabels(df_rrs['route'], fontsize=9)
ax.invert_yaxis()

# Add title and labels
ax.set_title('Rolling Relative Route Scoring System (R³S²)', ha='right',
             fontsize=20, fontweight='bold', pad=20, color='#2C3E50')
ax.set_xlabel('Performance Score (Higher = Faster Route)', fontsize=12, fontweight='bold', color='#34495E')
ax.set_ylabel('Route', fontsize=12, fontweight='bold', color='#34495E')

# Add subtitle with context
fig = plt.gcf()
fig.text(0.5, 0.96, f'10-Day Rolling Analysis • {len(df_rrs)} Routes Compared', 
         ha='center', fontsize=13, style='italic', color='#7F8C8D')

# Add grid for easier reading
ax.grid(axis='x', alpha=0.3, linestyle='--', linewidth=0.5)
ax.set_axisbelow(True)

# Add a vertical line at zero
ax.axvline(x=0, color='black', linewidth=1, linestyle='-', alpha=0.5)

# Style the plot
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_linewidth(0.5)
ax.spines['bottom'].set_linewidth(0.5)

# Add a color legend
from matplotlib.patches import Rectangle
legend_x = 0.02
for i, label in enumerate(['Fastest Routes', 'Average Routes', 'Slowest Routes']):
    color_idx = int(i * (len(colors) - 1) / 2)
    fig.text(legend_x + i*0.15, 0.02, label, fontsize=10,
             bbox=dict(boxstyle='round,pad=0.4', facecolor=colors[color_idx], alpha=0.8))

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

In [17]:
plot_traffic_square(df, days_offset=1.5, label='full')

,year,month,day,hour,route_code,label_full,label_short,duration,distance,avg_speed,ts,route_label
2226,2025,10,8,9,VJRQ+2M|RMJJ+F4,Kudlu Gate Metro Station → Biocon Campus,Hosur Road,28,10.3,22.07,2025-10-08 09:00:00,Kudlu Gate Metro Station → Biocon Campus
2227,2025,10,8,9,WH5F+26|WJ8X+F5W,Jaya Prakash Nagar Metro Station → Hemavathi P...,Double Decker Flyover,34,10.2,18.00,2025-10-08 09:00:00,Jaya Prakash Nagar Metro Station → Hemavathi P...
2228,2025,10,8,9,XJPW+92|WJP4+FF,Swami Vivekananda Road Metro Station → Christ ...,East Inner Ring,39,11.0,16.92,2025-10-08 09:00:00,Swami Vivekananda Road Metro Station → Christ ...
2229,2025,10,8,9,WHCJ+26|XGCP+FV,"RV Road Metro Station, Jayanagar 5th Block → V...",South Outer Ring,39,11.0,16.92,2025-10-08 09:00:00,"RV Road Metro Station, Jayanagar 5th Block → V..."
2230,2025,10,8,9,XPC7+72|XM33+J3,The Rameshwaram Cafe @ Brookfield → Gawky Goos...,Old Airport Road,42,10.6,15.14,2025-10-08 09:00:00,The Rameshwaram Cafe @ Brookfield → Gawky Goos...
...,...,...,...,...,...,...,...,...,...,...,...,...
2485,2025,10,9,18,WH5F+26|WJ8X+F5W,Jaya Prakash Nagar Metro Station → Hemavathi P...,Double Decker Flyover,42,10.2,14.57,2025-10-09 18:00:00,Jaya Prakash Nagar Metro Station → Hemavathi P...
2486,2025,10,9,18,2HVW+G8|XJXR+WG,"Bethel AG Church, Hebbal → SMVT Railway Station",North Outer Ring,41,9.9,14.49,2025-10-09 18:00:00,"Bethel AG Church, Hebbal → SMVT Railway Station"
2487,2025,10,9,18,WHCJ+26|XGCP+FV,"RV Road Metro Station, Jayanagar 5th Block → V...",South Outer Ring,46,10.2,13.30,2025-10-09 18:00:00,"RV Road Metro Station, Jayanagar 5th Block → V..."
2488,2025,10,9,18,XJPW+92|WJP4+FF,Swami Vivekananda Road Metro Station → Christ ...,East Inner Ring,53,11.3,12.79,2025-10-09 18:00:00,Swami Vivekananda Road Metro Station → Christ ...
